In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from pytorch_metric_learning.losses import SupConLoss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------------- DATASET ----------------
class EMGDataset(Dataset):
    def __init__(self, x_path, y_path):
        self.X = np.load(x_path)
        self.y = np.argmax(np.load(y_path), axis=1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.X[idx], dtype=torch.float32),
            torch.tensor(self.y[idx], dtype=torch.long)
        )

# ---------------- ENCODER ----------------
class CNNEncoder(nn.Module):
    def __init__(self, emb_dim=256):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv1d(6, 64, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=10),
            nn.ReLU(),
            nn.MaxPool1d(3),

            nn.Conv1d(64, 256, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=10),
            nn.ReLU(),

            nn.AdaptiveAvgPool1d(1)
        )

        self.fc = nn.Linear(256, emb_dim)

    def forward(self, x):
        x = x.permute(0, 2, 1)      # (B, C, T)
        x = self.features(x).squeeze(-1)
        z = self.fc(x)
        return z                   # (B, emb_dim)

# ---------------- CLASSIFIER ----------------
class Classifier(nn.Module):
    def __init__(self, emb_dim=256, num_classes=62):
        super().__init__()
        self.fc = nn.Linear(emb_dim, num_classes)

    def forward(self, z):
        return self.fc(z)

# ---------------- FULL MODEL ----------------
class FullModel(nn.Module):
    def __init__(self, emb_dim=256, num_classes=62):
        super().__init__()
        self.encoder = CNNEncoder(emb_dim)
        self.classifier = Classifier(emb_dim, num_classes)

    def forward(self, x):
        z = self.encoder(x)
        logits = self.classifier(z)
        return z, logits

# ---------------- EARLY STOPPING ----------------
class EarlyStopping:
    def __init__(self, patience=10):
        self.patience = patience
        self.best_acc = -1
        self.counter = 0
        self.best_state = None

    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.best_state = {k: v.cpu() for k, v in model.state_dict().items()}
            self.counter = 0
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

    def restore(self, model):
        model.load_state_dict(self.best_state)

# ---------------- TRAIN + EVAL ----------------
def train_and_eval(model, train_loader, test_loader,
                   supcon_loss, alpha=1.0,
                   epochs=500, patience=10):

    ce_loss = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=5e-4)
    stopper = EarlyStopping(patience)

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            z, logits = model(x)

            z_norm = F.normalize(z, dim=1)   # 🔥 REQUIRED for SupCon

            loss = ce_loss(logits, y) + alpha * supcon_loss(z_norm, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # ---- EVAL ----
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                _, logits = model(x)
                pred = logits.argmax(1)
                correct += (pred == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1:02d} | Val Acc: {acc:.4f}")

        if stopper.step(acc, model):
            print("Early stopping")
            break

    stopper.restore(model)
    return stopper.best_acc

BASE = "models/Data/Data/62_classes/UserIndependent"
supcon_loss = SupConLoss(temperature=0.1)

all_accs = []

for split in range(1, 20):
    print(f"\n===== SPLIT {split} =====")

    train_ds = EMGDataset(
        f"{BASE}/Train/X_train_{split}.npy",
        f"{BASE}/Train/y_train_{split}.npy"
    )
    test_ds = EMGDataset(
        f"{BASE}/Test/X_test_{split}.npy",
        f"{BASE}/Test/y_test_{split}.npy"
    )

    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=128, shuffle=False)

    model = FullModel(emb_dim=256, num_classes=62).to(device)

    acc = train_and_eval(
        model,
        train_loader,
        test_loader,
        supcon_loss,
        alpha=1.0,
        epochs=500,
        patience=10
    )

    print(f"Split {split} Accuracy: {acc:.4f}")
    all_accs.append(acc)

print("\nFINAL AVERAGE ACCURACY:", np.mean(all_accs))


Device: cuda

===== SPLIT 1 =====
Epoch 01 | Val Acc: 0.1032
Epoch 02 | Val Acc: 0.3161
Epoch 03 | Val Acc: 0.4403
Epoch 04 | Val Acc: 0.4855
Epoch 05 | Val Acc: 0.5887
Epoch 06 | Val Acc: 0.6387
Epoch 07 | Val Acc: 0.6484
Epoch 08 | Val Acc: 0.6484
Epoch 09 | Val Acc: 0.6145
Epoch 10 | Val Acc: 0.6484
Epoch 11 | Val Acc: 0.6323
Epoch 12 | Val Acc: 0.6435
Epoch 13 | Val Acc: 0.6065
Epoch 14 | Val Acc: 0.6597
Epoch 15 | Val Acc: 0.6548
Epoch 16 | Val Acc: 0.6419
Epoch 17 | Val Acc: 0.6452
Epoch 18 | Val Acc: 0.6226
Epoch 19 | Val Acc: 0.6306
Epoch 20 | Val Acc: 0.6145
Epoch 21 | Val Acc: 0.6403
Epoch 22 | Val Acc: 0.6726
Epoch 23 | Val Acc: 0.6387
Epoch 24 | Val Acc: 0.6532
Epoch 25 | Val Acc: 0.6468
Epoch 26 | Val Acc: 0.6565
Epoch 27 | Val Acc: 0.6355
Epoch 28 | Val Acc: 0.6403
Epoch 29 | Val Acc: 0.6419
Epoch 30 | Val Acc: 0.6339
Epoch 31 | Val Acc: 0.6339
Epoch 32 | Val Acc: 0.6226
Early stopping
Split 1 Accuracy: 0.6726

===== SPLIT 2 =====
Epoch 01 | Val Acc: 0.0548
Epoch 02 | Val

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

# ---------------- Setup ----------------
torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

BASE = "models/Data/Data/62_classes/UserIndependent"
NUM_CLASSES = 62
BATCH_SIZE = 128

class EMGDataset(Dataset):
    def __init__(self, x_path, y_path):
        X = np.load(x_path)
        y = np.argmax(np.load(y_path), axis=1)

        # Channel-wise normalization
        for i in range(X.shape[0]):
            for c in range(6):
                X[i, :, c] = (X[i, :, c] - X[i, :, c].mean()) / (X[i, :, c].std() + 1e-8)

        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class CNN_BiLSTM(nn.Module):
    def __init__(self, num_classes=62, emb_dim=256):
        super().__init__()

        # CNN Feature Extractor
        self.cnn = nn.Sequential(
            nn.Conv1d(6, 64, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=10),
            nn.ReLU(),
            nn.MaxPool1d(3),

            nn.Conv1d(64, 256, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=10),
            nn.ReLU()
        )

        # BiLSTM
        self.bilstm = nn.LSTM(
            input_size=256,
            hidden_size=128,
            batch_first=True,
            bidirectional=True
        )

        self.embedding = nn.Linear(256, emb_dim)
        self.classifier = nn.Linear(emb_dim, num_classes)

    def forward(self, x):
        x = x.permute(0, 2, 1)          # (B, C, T)
        x = self.cnn(x)                 # (B, 256, T')
        x = x.permute(0, 2, 1)          # (B, T', 256)

        lstm_out, _ = self.bilstm(x)
        feat = lstm_out[:, -1, :]       # last timestep

        emb = F.normalize(self.embedding(feat), dim=1)
        logits = self.classifier(emb)

        return emb, logits

class SupConLoss(nn.Module):
    def __init__(self, temperature=0.1):
        super().__init__()
        self.temperature = temperature

    def forward(self, z, y):
        z = F.normalize(z, dim=1)
        sim = torch.matmul(z, z.T) / self.temperature

        y = y.view(-1, 1)
        mask = (y == y.T).float()

        # remove self-comparisons
        logits_mask = torch.ones_like(mask) - torch.eye(z.size(0), device=z.device)
        mask = mask * logits_mask

        exp_sim = torch.exp(sim) * logits_mask
        log_prob = sim - torch.log(exp_sim.sum(dim=1, keepdim=True) + 1e-8)

        mean_log_prob_pos = (mask * log_prob).sum(dim=1) / (mask.sum(dim=1) + 1e-8)

        loss = -mean_log_prob_pos.mean()
        return loss

class EarlyStopping:
    def __init__(self, patience=8):
        self.patience = patience
        self.best = -1
        self.counter = 0
        self.best_state = None

    def step(self, acc, model):
        if acc > self.best:
            self.best = acc
            self.counter = 0
            self.best_state = {
                k: v.detach().cpu().clone()
                for k, v in model.state_dict().items()
            }
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

    def restore(self, model):
        model.load_state_dict(self.best_state)

def train_stage1(model, train_loader, test_loader, epochs=500):
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=5e-4)
    stopper = EarlyStopping(patience=8)

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            _, logits = model(x)
            loss = criterion(logits, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        correct = total = 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                _, logits = model(x)
                pred = logits.argmax(1)
                correct += (pred == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"[Stage-1] Epoch {epoch+1:02d} | Acc: {acc:.4f}")

        if stopper.step(acc, model):
            break

    stopper.restore(model)

def train_stage2(model, train_loader, test_loader, epochs=50):
    criterion = SupConLoss(temperature=0.1)

    # Freeze classifier
    for p in model.classifier.parameters():
        p.requires_grad = False

    optimizer = Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=1e-4
    )

    stopper = EarlyStopping(patience=6)

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            z, _ = model(x)
            loss = criterion(z, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        correct = total = 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                _, logits = model(x)
                pred = logits.argmax(1)
                correct += (pred == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"[Stage-2] Epoch {epoch+1:02d} | Acc: {acc:.4f}")

        if stopper.step(acc, model):
            break

    stopper.restore(model)

all_accs = []

for split in range(1, 20):
    print(f"\n===== SPLIT {split} =====")

    train_ds = EMGDataset(
        f"{BASE}/Train/X_train_{split}.npy",
        f"{BASE}/Train/y_train_{split}.npy"
    )
    test_ds = EMGDataset(
        f"{BASE}/Test/X_test_{split}.npy",
        f"{BASE}/Test/y_test_{split}.npy"
    )

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

    model = CNN_BiLSTM(num_classes=NUM_CLASSES).to(device)

    train_stage1(model, train_loader, test_loader)
    train_stage2(model, train_loader, test_loader)

    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            _, logits = model(x)
            pred = logits.argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)

    acc = correct / total
    print(f"Final Acc (Split {split}): {acc:.4f}")
    all_accs.append(acc)

    del model
    torch.cuda.empty_cache()

print("\nFINAL AVERAGE ACCURACY:", np.mean(all_accs))




Using device: cuda

===== SPLIT 1 =====
[Stage-1] Epoch 01 | Acc: 0.0645
[Stage-1] Epoch 02 | Acc: 0.0677
[Stage-1] Epoch 03 | Acc: 0.0952
[Stage-1] Epoch 04 | Acc: 0.1452
[Stage-1] Epoch 05 | Acc: 0.1032
[Stage-1] Epoch 06 | Acc: 0.1774
[Stage-1] Epoch 07 | Acc: 0.1839
[Stage-1] Epoch 08 | Acc: 0.2339
[Stage-1] Epoch 09 | Acc: 0.2855
[Stage-1] Epoch 10 | Acc: 0.2532
[Stage-1] Epoch 11 | Acc: 0.3048
[Stage-1] Epoch 12 | Acc: 0.3597
[Stage-1] Epoch 13 | Acc: 0.3903
[Stage-1] Epoch 14 | Acc: 0.4081
[Stage-1] Epoch 15 | Acc: 0.4694
[Stage-1] Epoch 16 | Acc: 0.4968
[Stage-1] Epoch 17 | Acc: 0.4452
[Stage-1] Epoch 18 | Acc: 0.4694
[Stage-1] Epoch 19 | Acc: 0.4887
[Stage-1] Epoch 20 | Acc: 0.4839
[Stage-1] Epoch 21 | Acc: 0.5468
[Stage-1] Epoch 22 | Acc: 0.4806
[Stage-1] Epoch 23 | Acc: 0.5468
[Stage-1] Epoch 24 | Acc: 0.5661
[Stage-1] Epoch 25 | Acc: 0.5484
[Stage-1] Epoch 26 | Acc: 0.5355
[Stage-1] Epoch 27 | Acc: 0.5694
[Stage-1] Epoch 28 | Acc: 0.5839
[Stage-1] Epoch 29 | Acc: 0.5839
[St

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from pytorch_metric_learning.losses import SupConLoss

# ---------------- Setup ----------------
torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------------- DATASET ----------------
class EMGDataset(Dataset):
    def __init__(self, x_path, y_path):
        X = np.load(x_path)
        y = np.argmax(np.load(y_path), axis=1)

        # channel-wise normalization (MATCHES KERAS)
        for i in range(X.shape[0]):
            for c in range(6):
                X[i, :, c] = (X[i, :, c] - X[i, :, c].mean()) / (X[i, :, c].std() + 1e-8)

        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ---------------- CNN + BiLSTM ENCODER ----------------
class CNN_BiLSTM_Encoder(nn.Module):
    def __init__(self, emb_dim=256):
        super().__init__()

        # CNN (same as your Keras)
        self.cnn = nn.Sequential(
            nn.Conv1d(6, 64, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=10),
            nn.ReLU(),
            nn.MaxPool1d(3),

            nn.Conv1d(64, 256, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=10),
            nn.ReLU()
        )

        # BiLSTM (same idea as Keras)
        self.bilstm = nn.LSTM(
            input_size=256,
            hidden_size=128,
            batch_first=True,
            bidirectional=True
        )

        self.fc = nn.Linear(256, emb_dim)

    def forward(self, x):
        x = x.permute(0, 2, 1)        # (B, C, T)
        x = self.cnn(x)               # (B, 256, T')
        x = x.permute(0, 2, 1)        # (B, T', 256)

        lstm_out, _ = self.bilstm(x)  # (B, T', 256)
        feat = lstm_out[:, -1, :]     # last timestep

        z = self.fc(feat)             # (B, emb_dim)
        return z

# ---------------- CLASSIFIER ----------------
class Classifier(nn.Module):
    def __init__(self, emb_dim=256, num_classes=62):
        super().__init__()
        self.fc = nn.Linear(emb_dim, num_classes)

    def forward(self, z):
        return self.fc(z)

# ---------------- FULL MODEL ----------------
class FullModel(nn.Module):
    def __init__(self, emb_dim=256, num_classes=62):
        super().__init__()
        self.encoder = CNN_BiLSTM_Encoder(emb_dim)
        self.classifier = Classifier(emb_dim, num_classes)

    def forward(self, x):
        z = self.encoder(x)
        logits = self.classifier(z)
        return z, logits

# ---------------- EARLY STOPPING ----------------
class EarlyStopping:
    def __init__(self, patience=10):
        self.patience = patience
        self.best_acc = -1
        self.counter = 0
        self.best_state = None

    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.best_state = {k: v.detach().cpu() for k, v in model.state_dict().items()}
            self.counter = 0
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

    def restore(self, model):
        model.load_state_dict(self.best_state)

# ---------------- TRAIN + EVAL ----------------
def train_and_eval(model, train_loader, test_loader,
                   supcon_loss, alpha=1.0,
                   epochs=500, patience=10):

    ce_loss = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=5e-4)
    stopper = EarlyStopping(patience)

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            z, logits = model(x)
            z = F.normalize(z, dim=1)  # REQUIRED FOR SUPCON

            loss = ce_loss(logits, y) + alpha * supcon_loss(z, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # -------- Validation --------
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                _, logits = model(x)
                pred = logits.argmax(1)
                correct += (pred == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1:03d} | Val Acc: {acc:.4f}")

        if stopper.step(acc, model):
            print("Early stopping")
            break

    stopper.restore(model)
    return stopper.best_acc

# ---------------- MAIN LOOP ----------------
BASE = "models/Data/Data/62_classes/UserIndependent"
supcon_loss = SupConLoss(temperature=0.1)

all_accs = []

for split in range(1, 20):
    print(f"\n===== SPLIT {split} =====")

    train_ds = EMGDataset(
        f"{BASE}/Train/X_train_{split}.npy",
        f"{BASE}/Train/y_train_{split}.npy"
    )
    test_ds = EMGDataset(
        f"{BASE}/Test/X_test_{split}.npy",
        f"{BASE}/Test/y_test_{split}.npy"
    )

    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=128)

    model = FullModel(emb_dim=256, num_classes=62).to(device)

    acc = train_and_eval(
        model,
        train_loader,
        test_loader,
        supcon_loss,
        alpha=1.0,
        epochs=500,
        patience=10
    )

    print(f"Split {split} Accuracy: {acc:.4f}")
    all_accs.append(acc)

    del model
    torch.cuda.empty_cache()

print("\nFINAL AVERAGE ACCURACY:", np.mean(all_accs))


Device: cuda

===== SPLIT 1 =====
Epoch 001 | Val Acc: 0.0161
Epoch 002 | Val Acc: 0.0597
Epoch 003 | Val Acc: 0.0887
Epoch 004 | Val Acc: 0.1274
Epoch 005 | Val Acc: 0.2274
Epoch 006 | Val Acc: 0.3371
Epoch 007 | Val Acc: 0.3484
Epoch 008 | Val Acc: 0.3823
Epoch 009 | Val Acc: 0.5258
Epoch 010 | Val Acc: 0.5516
Epoch 011 | Val Acc: 0.5452
Epoch 012 | Val Acc: 0.5242
Epoch 013 | Val Acc: 0.5500
Epoch 014 | Val Acc: 0.5952
Epoch 015 | Val Acc: 0.5871
Epoch 016 | Val Acc: 0.5806
Epoch 017 | Val Acc: 0.5871
Epoch 018 | Val Acc: 0.6371
Epoch 019 | Val Acc: 0.6032
Epoch 020 | Val Acc: 0.5790
Epoch 021 | Val Acc: 0.5629
Epoch 022 | Val Acc: 0.5790
Epoch 023 | Val Acc: 0.5613
Epoch 024 | Val Acc: 0.5887
Epoch 025 | Val Acc: 0.5677
Epoch 026 | Val Acc: 0.6048
Epoch 027 | Val Acc: 0.6065
Epoch 028 | Val Acc: 0.5790
Early stopping
Split 1 Accuracy: 0.6371

===== SPLIT 2 =====
Epoch 001 | Val Acc: 0.0242
Epoch 002 | Val Acc: 0.0613
Epoch 003 | Val Acc: 0.0629
Epoch 004 | Val Acc: 0.1613
Epoch 005